# 이진 판단


## 펄서 여부 판정 신경망


### 펄서 스타 데이터
펄서는 전파를 생성하는 중성자 별로 8개의 변수와 이진 클래스를 갖는 타겟 변수를 갖고 있습니다.

In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00372/HTRU2.zip

--2021-01-29 10:20:22--  https://archive.ics.uci.edu/ml/machine-learning-databases/00372/HTRU2.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1563015 (1.5M) [application/x-httpd-php]
Saving to: ‘HTRU2.zip’

HTRU2.zip           100%[===================>]   1.49M  3.81MB/s    in 0.4s    

2021-01-29 10:20:23 (3.81 MB/s) - ‘HTRU2.zip’ saved [1563015/1563015]



In [2]:
!unzip HTRU2.zip

Archive:  HTRU2.zip
  inflating: HTRU_2.csv              
  inflating: HTRU_2.arff             
  inflating: Readme.txt              


### 파이썬 모듈 불러들이기

In [6]:
import numpy as np
import csv
import time

np.random.seed(1234)

def randomize():
    np.random.seed(time.time())

### 하이퍼파라미터값의 정의

In [10]:
# 정규분포 난수값의 평균과 표준편차 정의
RND_MEAN = 0
RND_STD = 0.0030

# 학습률 정의
LEARNING_RATE = 0.001

### 파라미터 초기화 함수 정의

In [11]:
def init_model():
    global weight, bias, input_cnt, output_cnt
    weight = np.random.normal(RND_MEAN, RND_STD,[input_cnt, output_cnt])
    bias = np.zeros([output_cnt])

### 학습 및 평가 함수 정의

In [12]:
def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data()
    
    for epoch in range(epoch_count):
        losses, accs = [], []
        
        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)
            
        if report > 0 and (epoch+1) % report == 0:
            acc = run_test(test_x, test_y)
            print('Epoch {}: loss={:5.3f}, accuracy={:5.3f}/{:5.3f}'. \
                  format(epoch+1, np.mean(losses), np.mean(accs), acc))
            
    final_acc = run_test(test_x, test_y)
    print('\nFinal Test: final accuracy = {:5.3f}'.format(final_acc))

### 학습 및 평가  데이터 획득 함수 정의

In [13]:
def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)
    step_count = int(data.shape[0] * 0.8) // mb_size
    test_begin_idx = step_count * mb_size
    return step_count

def get_test_data():
    global data, shuffle_map, test_begin_idx, output_cnt
    test_data = data[shuffle_map[test_begin_idx:]]
    return test_data[:, :-output_cnt], test_data[:, -output_cnt:]

def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, output_cnt
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_idx])
    train_data = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]]
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]

### 학습 실행 함수와 평가 실행 함수 정의

In [15]:
def run_train(x, y):
    output, aux_nn = forward_neuralnet(x)
    loss, aux_pp = forward_postproc(output, y)
    accuracy = eval_accuracy(output, y)
    
    G_loss = 1.0
    G_output = backprop_postproc(G_loss, aux_pp)
    backprop_neuralnet(G_output, aux_nn)
    
    return loss, accuracy

def run_test(x, y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    return accuracy

### 단층 퍼셉트론에 대한 순전파 및 역전파 함수 정의

In [ ]:
def forward_neuralnet(x):
    global weight, bias
    output = np.matmul(x, weight) + bias
    return output, x

def backprop_neuralnet(G_output, x):
    global weight, bias
    g_output_w = x.transpose()
    
    G_w = np.matmul(g_output_w, G_output)
    G_b = np.sum(G_output, axis=0)

    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b

### 메인 함수 정의

In [23]:
def pulsar_exec(epoch_count=10, mb_size=10, report=1):
    load_pulsar_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report)

### 데이터 적재 함수의 정의

In [9]:
def load_pulsar_dataset():
    with open('HTRU_2.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        rows = []
        for row in csvreader:
            rows.append(row)

    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 8, 1
    data = np.asarray(rows, dtype='float32')

### 후처리 과정에 대한 순전파와 역전파 함수 재정의

In [16]:
def forward_postproc(output, y):
    entropy = sigmoid_cross_entropy_with_logits(y, output)
    loss = np.mean(entropy)
    return loss, [y, output, entropy]

def backprop_postproc(G_loss, aux):
    y, output, entropy = aux

    g_loss_entropy = 1.0 / np.prod(entropy.shape)
    g_entropy_output = sigmoid_cross_entropy_with_logits_derv(y, output)

    G_entropy = g_loss_entropy * G_loss
    G_output = g_entropy_output * G_entropy
    
    return G_output

### 정확도 계산 함수의 정의

In [17]:
def eval_accuracy(output, y):
    estimate = np.greater(output, 0)
    answer = np.greater(y, 0.5)
    correct = np.equal(estimate, answer)

    return np.mean(correct)

### 시그모이드 관련 함수 정의

In [18]:
def relu(x):
    return np.maximum(x, 0)

def sigmoid(x):
    return np.exp(-relu(-x)) / (1.0 + np.exp(-np.abs(x)))

def sigmoid_derv(x, y):
    return y * (1 - y)

def sigmoid_cross_entropy_with_logits(z, x):
    return relu(x) - x * z + np.log(1 + np.exp(-np.abs(x)))

def sigmoid_cross_entropy_with_logits_derv(z, x):
    return -z + sigmoid(x)

## 실행하기

### 펄서 여부 판정 신경망 실행하기

In [24]:
pulsar_exec()

Epoch 1: loss=0.145, accuracy=0.960/0.972
Epoch 2: loss=0.127, accuracy=0.967/0.974
Epoch 3: loss=0.134, accuracy=0.967/0.972
Epoch 4: loss=0.134, accuracy=0.968/0.969
Epoch 5: loss=0.132, accuracy=0.967/0.975
Epoch 6: loss=0.126, accuracy=0.969/0.972
Epoch 7: loss=0.126, accuracy=0.969/0.969
Epoch 8: loss=0.120, accuracy=0.970/0.974
Epoch 9: loss=0.121, accuracy=0.969/0.969
Epoch 10: loss=0.128, accuracy=0.969/0.969

Final Test: final accuracy = 0.969


## 확장하기

### 메인 실행 함수의 재정의

In [25]:
def pulsar_exec(epoch_count=10, mb_size=10, report=1, adjust_ratio=False):
    load_pulsar_dataset(adjust_ratio)
    init_model()
    train_and_test(epoch_count, mb_size, report)

### 데이터 적재 함수의 재정의

In [26]:
def load_pulsar_dataset(adjust_ratio):
    pulsars, stars = [], []
    with open('HTRU_2.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        rows = []
        for row in csvreader:
            if row[8] == '1': pulsars.append(row)
            else: stars.append(row)
            
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 8, 1
    
    star_cnt, pulsar_cnt = len(stars), len(pulsars)

    if adjust_ratio:
        data = np.zeros([2*star_cnt, 9])
        data[0:star_cnt, :] = np.asarray(stars, dtype='float32')
        for n in range(star_cnt):
            data[star_cnt+n] = np.asarray(pulsars[n % pulsar_cnt], dtype='float32')
    else:
        data = np.zeros([star_cnt+pulsar_cnt, 9])
        data[0:star_cnt, :] = np.asarray(stars, dtype='float32')
        data[star_cnt:, :] = np.asarray(pulsars, dtype='float32')

### 정확도 계산 함수의 재정의

In [27]:
def eval_accuracy(output, y):
    est_yes = np.greater(output, 0)
    ans_yes = np.greater(y, 0.5)
    est_no = np.logical_not(est_yes)
    ans_no = np.logical_not(ans_yes)
    
    tp = np.sum(np.logical_and(est_yes, ans_yes))
    fp = np.sum(np.logical_and(est_yes, ans_no))
    fn = np.sum(np.logical_and(est_no, ans_yes))
    tn = np.sum(np.logical_and(est_no, ans_no))
    
    accuracy = safe_div(tp+tn, tp+tn+fp+fn)
    precision = safe_div(tp, tp+fp)
    recall = safe_div(tp, tp+fn)
    f1 = 2 * safe_div(recall*precision, recall+precision)
    
    return [accuracy, precision, recall, f1]

def safe_div(p, q):
    p, q = float(p), float(q)
    if np.abs(q) < 1.0e-20: return np.sign(p)
    return p / q

### 출력문 수정을 위한 실행 함수 재정의

In [28]:
def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data()
    
    for epoch in range(epoch_count):
        losses = []
        
        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, _ = run_train(train_x, train_y)
            losses.append(loss)
            
        if report > 0 and (epoch+1) % report == 0:
            acc = run_test(test_x, test_y)
            acc_str = ','.join(['%5.3f']*4) % tuple(acc)
            print('Epoch {}: loss={:5.3f}, result={}'. \
                  format(epoch+1, np.mean(losses), acc_str))
            
    acc = run_test(test_x, test_y)
    acc_str = ','.join(['%5.3f']*4) % tuple(acc)
    print('\nFinal Test: final result = {}'.format(acc_str))

## 재정 함수 실행하기

### 원래 데이터셋으로 신경망 학습시키기

In [29]:
pulsar_exec()

Epoch 1: loss=0.136, result=0.972,0.879,0.794,0.835
Epoch 2: loss=0.124, result=0.973,0.959,0.732,0.830
Epoch 3: loss=0.127, result=0.966,0.990,0.629,0.770
Epoch 4: loss=0.129, result=0.973,0.970,0.717,0.824
Epoch 5: loss=0.126, result=0.961,0.752,0.841,0.794
Epoch 6: loss=0.134, result=0.973,0.911,0.769,0.834
Epoch 7: loss=0.130, result=0.975,0.949,0.757,0.842
Epoch 8: loss=0.132, result=0.972,0.955,0.723,0.823
Epoch 9: loss=0.120, result=0.975,0.953,0.754,0.842
Epoch 10: loss=0.123, result=0.971,0.982,0.692,0.812

Final Test: final result = 0.971,0.982,0.692,0.812


### 균형 잡힌 데이터로 신경망 학습시키기

In [30]:
pulsar_exec(adjust_ratio=True)

Epoch 1: loss=0.402, result=0.907,0.892,0.926,0.909
Epoch 2: loss=0.371, result=0.916,0.937,0.892,0.914
Epoch 3: loss=0.351, result=0.915,0.980,0.848,0.909
Epoch 4: loss=0.374, result=0.917,0.976,0.854,0.911
Epoch 5: loss=0.374, result=0.900,0.875,0.933,0.903
Epoch 6: loss=0.354, result=0.922,0.970,0.870,0.918
Epoch 7: loss=0.371, result=0.919,0.929,0.907,0.918
Epoch 8: loss=0.344, result=0.925,0.950,0.897,0.923
Epoch 9: loss=0.363, result=0.910,0.984,0.833,0.902
Epoch 10: loss=0.359, result=0.925,0.959,0.888,0.922

Final Test: final result = 0.925,0.959,0.888,0.922
